# Reporter Agent

This notebook illustrates how the very first version of this agent was developed

In [12]:
import sys
import json
from pathlib import Path
from dotenv import load_dotenv

from langgraph.graph import START, END, StateGraph
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.chat_models import init_chat_model

# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Load environment variables
env_path = project_root / '.env'
load_dotenv(dotenv_path=env_path)

print("✓ Imports successful")
print(f"✓ Loading .env from: {env_path}")


✓ Imports successful
✓ Loading .env from: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/.env


In [13]:
slug = "socotra_inside_the_alien_island"


## Prompts


In [5]:
# ============= FROZEN PROMPTS =============
# These prompts are frozen from the original implementation

magazine_profile = """ Agentic Newsroom is a magazine of discovery and wonder.
It explores the hidden corners of the planet and the universe, cutting-edge scientific mysteries, strange natural phenomena, ancient civilization, unusual places, and the latest in space exploration.
Our editorial voice blends scientific rigor, vivid storytelling, a touch of wonder, and strong narrative.
Articles are deeply researched, visually descriptive, and rich in detail.
We aim to make complex ideas accessible, without dumbing them down and to evoke awe, curiosity, and sense of scale.
"""

magazine_guardrails = """
Agentic Newsroom adheres to the following editorial standards in all published writing:

1. Agentic Newsroom does not produce or publish any form of hate speech, harassment, racism, sexism, or discriminatory content.
2. Articles may occasionally discuss pseudoscience, fringe theories, myths, or unverified claims when relevant to cultural or historical context. 
   However, Agentic Newsroom does not endorse such claims and always presents them with clear scientific, historical, or factual framing.
3. All information must be presented accurately, responsibly, and with respect for both scientific consensus and ethical journalism.
4. When uncertainty exists, Agentic Newsroom clearly distinguishes between established fact, informed interpretation, and speculation.
"""

reporter_profile = """
Your name is Jennifer. You are the lead reporter for Agentic Newsroom.
You are an accomplished magazine journalist with more than twenty years of experience in various high profile scientific and geographical magazines.
Your deeply research reporting blends scientific rigor, clear narrative structure, and vivid descriptive writing.
You are endlessly curious about the planet, the natural world, history, science, and remote places. 
You always strive to uncover the most compelling angle and explore overlooked details.
You prioritize clarity, accuracy, and factual integrity. 
You rely only on verified information and credible sources.
Your research is meticulous and method-driven. 
You organize complex information, verify claims, resolve contradictions, and present findings in a structured and engaging narrative.
Your job is to produce well-written, deeply researched articles that reflect Agentic Newsroom's editorial standards: 
authentic curiosity, intellectual honesty, and a sense of wonder about the world.
"""

reporter_write_draft_prompt = """You are a reporter tasked with writing an article based on a provided Story Brief and a Research Package.

<Magazine Profile>
This is the magazine you work for:
{magazine_profile}
</Magazine Profile>

<Reporter Profile>
This is your profile:
{reporter_profile}
</Reporter Profile>

<Story Brief>
This is the story brief you have been tasked to write:
{story_brief}
</Story Brief>

<Magazine Guardrails>
Strictly adhere to the following editorial standards:
{magazine_guardrails}
</Magazine Guardrails>

<Research Material>
You have access to a Research Package containing collected search results.
Each result includes the source URL and the content found.
Use this material as the factual basis for your article.
</Research Material>

{examples}

<Task>
Your task is to write a complete Draft Package.
This includes the full article text, a list of sources used, flagged facts that need verification, and structural notes.
</Task>

<Instructions>
1. **Analyze**: Read the Story Brief to understand the angle. Read the Research Material to gather facts.
2. **Plan**: Decide on the structure and flow of the article. Pay extra attention to the target length of the article.
3. **Write**: Compose the article in your own voice.
   - **Do NOT copy-paste** text from sources. Rewrite in your own words.
   - **Attribute** every major fact or claim to its source (e.g., "According to National Geographic...").
4. **Cite**: Keep track of which sources you use for the `sources` list.
5. **Verify**: Flag any claims that seem dubious or contradictory in the `flagged_facts` list.
</Instructions>

<CRITICAL>
- **Attribution**: You must attribute information to its source within the text.
- **No Direct Quotes**: Do not use direct quotes unless you are quoting a public figure or official statement found in the research. You cannot interview people yourself.
- **Originality**: The writing must be yours. The facts come from the research.
- **Length**: Pay careful attention that your article aims for the target length defined in the story brief.
</CRITICAL>
"""

print("✓ Prompts defined")

✓ Prompts defined


## States and Structured Outputs

In [6]:
# ============= FROZEN SCHEMAS =============
# These schemas are frozen from the original implementation

from typing import List, Optional, TypedDict
from pydantic import BaseModel, Field

class StoryBrief(BaseModel):
    """Story brief created by the Assignment Editor."""
    topic: str = Field(..., description="Clear statement of what the story is about")
    angle: str = Field(..., description="The specific approach or perspective to take")
    length: int = Field(..., description="The length of the article in words")
    key_questions: List[str] = Field(..., description="3-5 questions the article should answer")

class SearchResult(BaseModel):
    search_result: str = Field(..., description="The search result content")
    source: str = Field(..., description="The source of the search result")

class ResearchPackage(BaseModel):
    """Research package created by the Research Assistant."""
    results: List[SearchResult] = Field(..., description="List of collected search results")

class DraftPackage(BaseModel):
    """Draft package created by the Reporter."""
    full_draft: str = Field(..., description="The complete article text")
    sources: List[str] = Field(..., description="List of sources (URLs, publications, experts)")
    flagged_facts: List[str] = Field(..., description="Facts/claims that need verification")
    structural_notes: List[str] = Field(..., description="Notes about the draft structure")

class ReporterState(TypedDict):
    """State for the Reporter subgraph."""
    story_brief: StoryBrief
    research_package: ResearchPackage
    feedback: Optional[str]
    draft_package: Optional[DraftPackage]

## Tools and Configuration


In [7]:

model = init_chat_model(model="openai:gpt-5.1", reasoning_effort="medium")
# model = init_chat_model(model="openai:gpt-5-mini", reasoning_effort="minimal")

print("✓ Configuration complete")


✓ Configuration complete


## Nodes


In [8]:
def write_draft(state: ReporterState):
    # Get inputs from state
    story_brief = state["story_brief"]
    research_package = state["research_package"]
    
    # Format inputs for the prompt
    brief_str = story_brief.model_dump_json(indent=2)
    
    # Format research results into a readable string for the LLM
    # We can use the same formatting logic we used for the markdown report
    research_str = ""
    for i, item in enumerate(research_package.results, 1):
        research_str += f"Source {i}: {item.source}\nContent: {item.search_result}\n\n"


    # Prepare examples if available
    examples_str = ""
    examples_dir = project_root / "examples" / "articles"
    
    if examples_dir.exists():
        md_files = list(examples_dir.glob("*.md"))
        if md_files:
            examples_content = []
            for md_file in md_files:
                with open(md_file, "r") as f:
                    content = f.read()
                    examples_content.append(f"--- Example Article: {md_file.name} ---\n{content}\n")
            
            if examples_content:
                examples_str = "<Examples>\nHere are some example articles to guide your tone and style:\n\n" + "\n".join(examples_content) + "</Examples>\n"

    system_msg_content = reporter_write_draft_prompt.format(
        magazine_profile=magazine_profile,
        reporter_profile=reporter_profile,
        story_brief=brief_str,
        magazine_guardrails=magazine_guardrails,
        examples=examples_str 
    )
           
    
    # Pass research as a HumanMessage context
    messages = [
        SystemMessage(content=system_msg_content),
        HumanMessage(content=f"Here is the collected research material:\n\n{research_str}")
    ]
    
    structured_model = model.with_structured_output(DraftPackage)
    response = structured_model.invoke(messages)
    
    return {
        "draft_package": response
    }

print("✓ Nodes defined")


✓ Nodes defined


## Graph Building


In [9]:
builder = StateGraph(ReporterState)
builder.add_node("write_draft", write_draft)
builder.add_edge(START, "write_draft")
builder.add_edge("write_draft", END)

reporter_agent = builder.compile()

print("✓ Graph compiled")
print("\nGraph structure (ASCII):")
print(reporter_agent.get_graph().draw_ascii())


✓ Graph compiled

Graph structure (ASCII):
 +-----------+   
 | __start__ |   
 +-----------+   
        *        
        *        
        *        
+-------------+  
| write_draft |  
+-------------+  
        *        
        *        
        *        
  +---------+    
  | __end__ |    
  +---------+    


## Test Run


In [10]:
def load_story_brief(slug: str) -> StoryBrief:
    """Load a StoryBrief from the tmp directory."""
    path = project_root / "tmp" / slug / "story_brief.json"
    if not path.exists():
        raise FileNotFoundError(f"StoryBrief not found at {path}")
    
    with open(path, "r") as f:
        data = json.load(f)
    return StoryBrief(**data)

def load_research_package(slug: str) -> ResearchPackage:
    """Load a ResearchPackage from the tmp directory."""
    path = project_root / "tmp" / slug / "research_package.json"
    if not path.exists():
        raise FileNotFoundError(f"ResearchPackage not found at {path}")
    
    with open(path, "r") as f:
        data = json.load(f)
    return ResearchPackage(**data)

# Load inputs
story_brief = load_story_brief(slug)
research_package = load_research_package(slug)

print("✓ Story Brief Loaded")
print(f"  Topic: {story_brief.topic}")
print(f"  Key Questions: {len(story_brief.key_questions)}")
print(f"\n✓ Research Package Loaded")
print(f"  Total Research Items: {len(research_package.results)}")


✓ Story Brief Loaded
  Topic: Socotra: Inside the ‘Alien’ Island Where Evolution Went Off-Script
  Key Questions: 5

✓ Research Package Loaded
  Total Research Items: 27


In [14]:
result = reporter_agent.invoke({
    "story_brief": story_brief,
    "research_package": research_package
})

print("✓ Agent execution complete")

✓ Agent execution complete


In [15]:
from langchain_core.messages import AIMessage, HumanMessage
from IPython.display import display, Markdown

def format_draft_package_as_markdown(draft_package) -> str:
    """Format a DraftPackage as markdown."""
    md = []
    
    # Header
    md.append("# 📄 Draft Package")
    md.append("")
    md.append(f"**Draft Length:** {len(draft_package.full_draft)} characters")
    md.append(f"**Sources:** {len(draft_package.sources)}")
    md.append(f"**Flagged Facts:** {len(draft_package.flagged_facts)}")
    md.append(f"**Structural Notes:** {len(draft_package.structural_notes)}")
    md.append("")
    md.append("---")
    md.append("")
    
    # Full Draft
    md.append("## 📝 Full Draft")
    md.append("")
    md.append(draft_package.full_draft)
    md.append("")
    md.append("---")
    md.append("")
    
    # Sources
    md.append("## 📚 Sources")
    md.append("")
    if draft_package.sources:
        for i, source in enumerate(draft_package.sources, 1):
            md.append(f"{i}. {source}")
    else:
        md.append("*(No sources listed)*")
    md.append("")
    md.append("---")
    md.append("")
    
    # Flagged Facts
    md.append("## ⚠️ Flagged Facts")
    md.append("")
    if draft_package.flagged_facts:
        for i, fact in enumerate(draft_package.flagged_facts, 1):
            md.append(f"{i}. {fact}")
    else:
        md.append("*(No flagged facts)*")
    md.append("")
    md.append("---")
    md.append("")
    
    # Structural Notes
    md.append("## 📋 Structural Notes")
    md.append("")
    if draft_package.structural_notes:
        for i, note in enumerate(draft_package.structural_notes, 1):
            md.append(f"{i}. {note}")
    else:
        md.append("*(No structural notes)*")
    md.append("")
    
    return "\n".join(md)

# Display the draft package
if "draft_package" in result and result["draft_package"]:
    markdown_output = format_draft_package_as_markdown(result["draft_package"])
    display(Markdown(markdown_output))
else:
    # Fallback
    messages = result.get("messages", [])
    draft_message = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
            draft_message = msg
            break
    
    if draft_message:
        display(Markdown(f"# ⚠️ Draft (Fallback from Messages)\n\n{draft_message.content}"))
    else:
        print("⚠️ No draft found in output.")

# 📄 Draft Package

**Draft Length:** 8796 characters
**Sources:** 5
**Flagged Facts:** 3
**Structural Notes:** 6

---

## 📝 Full Draft

On the limestone plateau of Firmihin, the trees look like something sketched for a science‑fiction set and then mistakenly left on Earth. Their trunks rise straight and pale before exploding into tight, green umbrellas that cast perfectly circular shadows on the rock. When one is cut or wounded, a dark sap oozes from the bark and dries to the color of dried blood.

This is Dracaena cinnabari, the dragon’s blood tree—Socotra’s emblem and, to many visitors, proof that this island in the northwest Indian Ocean really does resemble another planet. According to a 2018 paper archived in the U.S. National Library of Medicine’s PMC database, the species is found only here and only between about 300 and 1,480 meters above sea level, tracking the belt of fog and higher moisture that clings to Socotra’s mountains and plateaus.

Socotra is a Yemeni island, but on the map it hangs alone, nearly 380 kilometers south of the Arabian Peninsula and about 232 kilometers east of the Horn of Africa, as summarized by Wikipedia’s entry on the island. Roughly 132 kilometers long and 42 kilometers wide, it makes up about 95 percent of the land in the Socotra Archipelago. The rest of the governorate, described in Yemen’s Socotra Governorate records and summarized on Wikipedia, is scattered in smaller pieces: Abd al Kuri, Samhah, Darsah, and a handful of rocky islets.

## A continent shard adrift

Socotra’s otherworldly feel begins with its geology. According to the Socotra Governorate overview on Wikipedia, the island is not volcanic, like Hawaii or the Galápagos, but a continental fragment—a sliver of ancient Gondwana that broke away from Africa. Geologists link its separation to Miocene‑age rifting that opened the Gulf of Aden, leaving Socotra as an isolated block of continental crust marooned in tropical seas.

That deep history set the stage for an evolutionary experiment. The same governorate sources describe an island built largely of limestone, lifted and carved into highlands by the Hajhir Mountains, where granite pokes through the carbonate cap. The lowlands are starkly arid, receiving around 150 millimeters of rainfall a year. Most of that comes with the northeast monsoon from November to March; from April to October the southwest monsoon brings fierce, dry winds.

Higher up, the pattern flips. Moist air off the sea condenses into fog, wrung out as it rises over the mountains. Wikipedia’s summary of the archipelago’s climate notes that this orographic mist can deliver up to 1,000 millimeters of moisture per year to the highlands, feeding narrow, perennial streams found only in the mountains. To a drought‑adapted tree like D. cinnabari, that band of cloud and drizzle is life itself.

## Where evolution went off‑script

Within that patchwork of foggy summits and sun‑blasted plains, evolution had time—millions of years of it—to improvise. Socotra’s main Wikipedia entry, drawing on botanical surveys, estimates that roughly one‑third of the island’s plant species occur nowhere else on Earth. A 2023 paper in the journal *Forests* refined that figure for vascular plants to about 37 percent, underscoring just how unusual Socotra’s flora has become.

Dragon’s blood trees are only the most famous expression of that isolation. The 2018 PMC article and subsequent *Forests* studies describe D. cinnabari as one of several arborescent, or tree‑forming, members of the Dracaena group, many of which are insular and threatened across their ranges. On Socotra, researchers have estimated a population of around 80,000 dragon’s blood trees, according to a 2022 *Forests* study that used field data and habitat modeling to locate suitable zones for conservation and restoration.

Those studies emphasize that the trees’ current distribution hugs the foggy mid‑ and high‑elevation zones, making them especially sensitive to shifts in cloud patterns or land use. To understand how these strange forests are changing, a team led by Czech researchers inventoried dragon’s blood trees on the Firmihin plateau between 2010 and 2011, as summarized in the 2023 *Forests* article. Working across roughly 700 hectares, they laid out more than a hundred circular plots, measured every tree taller than 1.3 meters, and mapped the distinctive multi‑branched crowns—establishing a baseline for future monitoring.

Botanists and biogeographers see the island as a natural laboratory: a continental shard that carried an ancient African flora out to sea, then left it to adapt to an extreme, wind‑scoured climate. The island’s Tertiary‑age separation from the mainland, noted in the PMC and *Forests* papers, helps explain why so many lineages on Socotra have diverged into their own species.

## An island of scripts and stories

Socotra may look alien, but it has been woven into human history for thousands of years. Archaeological work summarized on Wikipedia’s Socotra page reports evidence of an Oldowan‑style stone tool culture near the modern town of Hadibo, hinting at very ancient human presence. Later, the island appears in the *Periplus of the Erythraean Sea*, a Greek‑language guide to Indian Ocean trade from around the first century.

Inside Hoq Cave on Socotra’s north coast, explorers have documented inscriptions in multiple scripts, from South Arabian to Greek and Indian Brahmi, dating between the first century BCE and the sixth century CE, according to the same Wikipedia summary. Together, they suggest that sailors from around the region once sheltered under the island’s cliffs, carving their names and prayers into the cave walls while waiting for the monsoon winds to turn.

Today, the people who call the island home are known as Socotrans. As Wikipedia notes, most speak both Arabic and Soqotri, a Modern South Arabian language that, like the island’s flora, evolved in relative isolation.

Politically, Socotra has spent centuries tethered to distant powers. The Socotra Governorate history compiled on Wikipedia traces its passage through the Mahra Sultanate, a brief Portuguese occupation in the early 1500s, and later British control under the Bombay Presidency and then the Aden Protectorate. After South Yemen’s independence in 1967, Socotra was folded into Aden Governorate, and then Hadhramaut, before being designated its own governorate in 2013.

That administrative shuffling has continued in the twenty‑first century, often reflecting wider turmoil in Yemen. In April 2018, the United Arab Emirates deployed troops to Socotra and, according to the Socotra Governorate and Socotra pages, took administrative control of the island’s airport and seaport. Saudi forces arrived in May 2018 and brokered an agreement that restored formal control to Yemen’s government. In 2020, however, the UAE‑backed Southern Transitional Council, a secessionist group, seized effective control of the archipelago, a situation that, according to Wikipedia’s latest summaries, continues today.

## A fragile World Heritage

Socotra’s singular mix of geology, climate, and life did not go unnoticed by conservation bodies. The island was recognized as a UNESCO biosphere reserve in 2003, according to the 2023 *Forests* article, and inscribed as a World Heritage Site in 2008, as noted by Wikipedia. Those designations acknowledge not only the dragon’s blood forests but the full suite of endemic plants and the broader island landscape.

Scientists working on Socotra increasingly frame the archipelago as a test case for how isolated ecosystems might weather the twenty‑first century. Because dragon’s blood trees and many other endemic species are tightly linked to fog‑fed mountain habitats, any change in monsoon rhythms or cloud formation could redraw the map of where they can survive. The 2022 *Forests* modeling study, for example, sought to identify pockets of land where restoration would be most successful under changing conditions, while also pointing out that dragon tree species globally tend to be rare and endangered, often with small, insular populations.

At the same time, Socotra’s people are navigating the pressures of conflict, isolation, and the outside world’s growing fascination with their home. Reliable data on tourism and its impacts are scarce in the sources summarized here, but conservationists and local authorities will likely face a familiar dilemma seen in other remote World Heritage Sites: how to balance economic opportunities from visitors with the need to protect fragile habitats and respect local culture.

For now, the dragon’s blood trees still stand on their wind‑carved plateaus, catching the fog in their umbrella crowns. Their red resin once fed ancient trade routes and medicinal traditions; today, their survival may tell us how well an “alien” ecosystem can endure on an increasingly crowded and warming planet.


---

## 📚 Sources

1. https://en.wikipedia.org/wiki/Socotra
2. https://en.wikipedia.org/wiki/Socotra_Governorate
3. https://pmc.ncbi.nlm.nih.gov/articles/PMC6169433/
4. https://www.mdpi.com/1999-4907/13/8/1276
5. https://www.mdpi.com/1999-4907/14/4/840

---

## ⚠️ Flagged Facts

1. The statement that researchers have estimated around 80,000 dragon’s blood trees comes from the 2022 *Forests* article and should be cross‑checked against the most recent population assessments, as numbers may have been updated or revised.
2. The description of Socotra as a potential future tourism dilemma (balancing economic opportunity and habitat protection) is an extrapolation based on patterns at other World Heritage Sites; specific, up‑to‑date tourism statistics or documented impacts on Socotra are not provided in the current research package and should be verified before publication.
3. Characterizing Socotra as a ‘natural laboratory’ for climate resilience and twenty‑first‑century change is an interpretive framing based on the scientific literature’s emphasis on endemism, isolation, and modeling of habitat shifts; it is not a formal designation and should be kept as analytical language rather than presented as a direct claim by a specific institution.

---

## 📋 Structural Notes

1. The article opens with a vivid scene centered on the dragon’s blood trees and their resin to immediately evoke Socotra’s ‘alien’ reputation, in line with the brief’s suggested opening imagery.
2. Section 1 (“A continent shard adrift”) establishes geographic context (location, size, archipelago composition) and geologic/climatic background, grounding the later discussion of biodiversity in plate tectonics and monsoon dynamics.
3. Section 2 (“Where evolution went off‑script”) explains endemism and uses dragon’s blood trees and related research (population estimates, elevation band, monitoring methods) as a focal example of Socotra as a ‘living laboratory.’
4. Section 3 (“An island of scripts and stories”) brings in human history and culture: early stone‑tool evidence, Socotra’s role in ancient trade, cave inscriptions, languages, and the long arc of political control, leading into recent geopolitical tensions.
5. Section 4 (“A fragile World Heritage”) addresses formal conservation status (biosphere reserve, World Heritage), summarizes current scientific work on restoration and habitat modeling, and then widens to the broader tension between protection, conflict, and potential tourism pressures, foregrounding the central question of whether Socotra can remain ‘otherworldly’ in a changing world.
6. The piece is structured to hit the key questions from the brief: unique landscape/organisms (through descriptive opening and endemism section), geological/evolutionary history, local people and history, scientific research as a natural laboratory, and modern pressures including governance instability and anticipated conservation/tourism challenges.


In [16]:
import re
from pathlib import Path

def save_draft_package(state, project_root=Path.cwd().parent):
    """Saves the final draft package to the same slug folder as the StoryBrief."""
    
    # 1. Extract necessary data
    # Note: We rely on the global 'slug' variable being defined in the notebook
    if "slug" not in globals():
        print("⚠️ 'slug' variable not found. Cannot determine save path.")
        return

    # 2. Get the DraftPackage object
    draft_package = state.get("draft_package")
    
    if not draft_package:
        print("⚠️ No 'draft_package' found in state to save.")
        return
    
    # 3. Define paths
    tmp_dir = project_root / "tmp"
    article_dir = tmp_dir / slug
    
    # Ensure directory exists
    article_dir.mkdir(parents=True, exist_ok=True)
    
    # 4. Save Draft Package as JSON (Metadata + Content)
    json_path = article_dir / "draft_package.json"
    with open(json_path, "w", encoding="utf-8") as f:
        f.write(draft_package.model_dump_json(indent=2))
        
    # 5. Save Readable Markdown Draft using the same formatter
    md_path = article_dir / "draft_package.md"
    markdown_content = format_draft_package_as_markdown(draft_package)
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)
        
    print(f"✅ Draft package (JSON) saved to: {json_path}")
    print(f"✅ Draft package (Markdown) saved to: {md_path}")

# Run the save function
save_draft_package(result)

✅ Draft package (JSON) saved to: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/tmp/socotra_inside_the_alien_island/draft_package.json
✅ Draft package (Markdown) saved to: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/tmp/socotra_inside_the_alien_island/draft_package.md
